In [1]:
import pandas as pd
import os

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import SimilarityEvaluator, F1ScoreEvaluator, BleuScoreEvaluator, GleuScoreEvaluator, MeteorScoreEvaluator

In [2]:
from dotenv import load_dotenv
load_dotenv(".env")
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment":os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

In [3]:
import evalprompt.generateApiSpec as generateApiSpec
import os
current_folder = globals()['_dh'][0]
# Generate output from the test dataset, the jsonl file under this output will be used as the evaluation dataset for the prompt
query='Please be professional, and use below infomation to generate an OpenAPI specification documentation with YAML format:'
api_folder = os.path.join(current_folder, "./apisrc")
apispec_dir = os.path.join(current_folder, "./apispec")
output_dir = os.path.join(current_folder, "./output")
generateApiSpec.generateOutputData(query, api_folder, apispec_dir, output_dir)

In [ ]:
import random
randomNum = random.randint(1111, 9999)
similarity_evaluator = SimilarityEvaluator(model_config)
data_path = "./output/output.jsonl"
results = evaluate(
    evaluation_name="Eval-Run-" + str(randomNum),
    data=data_path,
    evaluators={
        "relevance": similarity_evaluator,
    },
    evaluator_config={
        "relevance": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.context}",
        }
    }
)

In [ ]:
glue_eval = GleuScoreEvaluator()
blue_eval = BleuScoreEvaluator()
f1_eval = F1ScoreEvaluator()
metreo_eval = MeteorScoreEvaluator()
data_path = "./output/output.jsonl"
calc_results = evaluate(
    data=data_path,
    evaluators = {
        "glue": glue_eval,
        'blue': blue_eval,
        'f1': f1_eval,
        'metreo': metreo_eval
    },
    evaluator_config={
        "glue": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        },
        "blue": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        },
        "f1": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        },
        "metreo": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        }
    }
)

In [28]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])

In [ ]:
pprint(calc_results)

In [6]:
pd.DataFrame(calc_results["rows"])

,inputs.response,inputs.context,inputs.ground_truth,outputs.glue.gleu_score,outputs.blue.bleu_score,outputs.f1.f1_score,outputs.metreo.meteor_score
0,```yaml\nopenapi: 3.0.3\ninfo:\n title: Weath...,"[ApiController]\n[Route(""[controller]"")]\npubl...",openapi: 3.0.0\ninfo:\n title: Weather Foreca...,0.648980,0.640381,0.832117,0.632723
1,```yaml\nopenapi: 3.0.0\ninfo:\n title: Pets ...,const express = require('express');\nconst app...,openapi: 3.0.0\ninfo:\n title: Pets API\n ve...,0.501836,0.492562,0.745000,0.491764
2,```yaml\nopenapi: 3.0.0\ninfo:\n title: Flask...,"from flask import Flask, jsonify, request \nfr...",openapi: 3.0.0\ninfo:\n title: API Specificat...,0.591139,0.578732,0.805825,0.555505
